In [184]:
#first iteration of the script after migration, combining information in a PDF with a zones table uploaded to the reports database
#CHALLENGE: report initially delivered in a PDF with complicated formatting
#saved at least 50% in catalog shipment costs, run several times a week

import tabula
import pandas as pd
import numpy as np
from functools import reduce

file_path = r'C:\\Users\\USER\\OneDrive - COMPANY\\Data Projects\\Catalogs\\catalogs.pdf'

list_original = tabula.read_pdf(file_path, pages = 'all')

import warnings

warnings.filterwarnings('ignore')

import pyodbc 

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

query = '''

SELECT *

FROM USPS_Zones

'''

dfUSPSZones = pd.read_sql(query, cnxn)

df_appended = []

df_appended = pd.concat(list_original)

df_appended.drop(df_appended.columns[[1,3,4,5,6,7,8,9,10,11,12,13,14]], axis = 1, inplace=True)

df_appended = df_appended.rename(columns={'Order No. Date & Time': 'Filter', 'Ship To Sold To': 'Address'})

df_appended = df_appended[df_appended.Filter == 'LEH - Catalog']

df_appended = df_appended.reset_index(drop=True)

df_appended.drop(df_appended.columns[[0]], axis = 1, inplace=True)

df_appended = df_appended[~df_appended['Address'].str.endswith('Canada')]

df_appended[['Name','Address1','Address2','Address3']] = df_appended['Address'].str.split('/',expand = True)

df_appended.drop(df_appended.columns[[0]], axis = 1, inplace=True)

df_appended.fillna('',inplace=True)

# conditional column

df_appended['Address4'] = np.where((df_appended['Address3'] == ''), df_appended['Address2'], df_appended['Address3'])

df_appended['Address2'] = np.where((df_appended['Address4'] == df_appended['Address2']), '', df_appended['Address2'])

# split the name

df_appended[['LastName','FirstName']] = df_appended['Name'].str.split(',',expand = True)

cols = df_appended.columns.tolist()

cols = cols[-1:] + cols[:-1]

cols = cols[-1:] + cols[:-1]

df_appended = df_appended[cols]

df_appended.drop(df_appended.columns[[2,5]], axis = 1, inplace=True)

df_appended = df_appended.rename(columns={'Address4': 'Address3'})

AddressList = df_appended['Address3'].str.rsplit('  ', 1)

df_AddressList = pd.DataFrame(AddressList)

df_AddressList[['CityState','ZIPCODE']] = pd.DataFrame(df_AddressList.Address3.tolist(), index= df_AddressList.index)

df_AddressList.drop(df_AddressList.columns[[0]], axis = 1, inplace=True)

AddressList2 = df_AddressList['CityState'].str.rsplit(' ', 1)

df_AddressList2 = pd.DataFrame(AddressList2)

df_AddressList2[['City','State']] = pd.DataFrame(df_AddressList2.CityState.tolist(), index= df_AddressList2.index)

df_AddressList2.drop(df_AddressList2.columns[[0]], axis = 1, inplace=True)

# compile the list of dataframes you want to merge
data_frames = [df_appended, df_AddressList, df_AddressList2]

df_merged = reduce(lambda left,right: pd.merge(left, right,left_index=True, right_index=True, how='left'), data_frames).fillna('')

df_merged.drop(df_merged.columns[[4,5]], axis = 1, inplace=True)

df_merged = df_merged.reindex(columns=['LastName', 'FirstName', 'Address1','Address2','City', 
                                       'State','ZIPCODE'])

dfJoined = df_merged.merge(dfUSPSZones, how = 'cross')

dfJoined['ZIPCODE_NUMERIC'] = dfJoined['ZIPCODE'].str.slice(0,5)

dfJoined[['ZipCode_FROM', 'ZipCode_TO', 'ZIPCODE_NUMERIC', 'Zone']] = dfJoined[['ZipCode_FROM', 'ZipCode_TO', 'ZIPCODE_NUMERIC', 'Zone']].apply(pd.to_numeric)

dfJoined = dfJoined.query('ZipCode_FROM <= ZIPCODE_NUMERIC <= ZipCode_TO')

dfJoined['Zone'] = dfJoined['Zone'].fillna(0)

dfJoined['Zone'] = dfJoined['Zone'].astype(int)

dfJoined.drop(['ZipCode_FROM', 'ZipCode_TO', 'ZIPCODE_NUMERIC'], inplace=True, axis=1)

dfJoined = dfJoined.rename({'Zone': 'ZONE'}, axis = 1)

dfJoined.reset_index(drop=True, inplace=True)

dfJoined.to_excel('WeeklyCatalogShipments.xlsx', index=False, header=True)

import shutil

original = r'C:\Users\USER\OneDrive - COMPANY\Code\Python\Jupyter\DataProjects\CatalogShipments\WeeklyCatalogShipments.xlsx'
target = r'C:\Users\USER\OneDrive - COMPANY\Data Projects\Catalogs\WeeklyCatalogShipments.xlsx'

shutil.copyfile(original, target)

print('Done.')

Done.
